# Workshop: Train a Medical AI Model in One Day
## Notebook 1: Dataset Setup & Visualization

Load local dataset in Fiftyone

In [ ]:
%pip install --upgrade pip
%pip install fiftyone umap-learn torch torchvision

In [ ]:
import fiftyone as fo

In [ ]:
# 🔧 Configuration
datasetUUID = "068818885c547b1e80002649e95254ce"
fiftyoneExportPath = "/Users/paularamos/Documents/GitHub/eyepop_ai/Voxel51-Medical-Collab/notebooks/.cache/voxel51/068818885c547b1e80002649e95254ce/fiftyone_dataset"

In [ ]:

name = f"eyepop_dataset_{datasetUUID}"
if name in fo.list_datasets():
    print(f"Dataset {name} already exists. Deleting it...")
    fo.delete_dataset(name)

dataset = fo.Dataset.from_dir(
    dataset_dir=fiftyoneExportPath,
    dataset_type=fo.types.FiftyOneDataset,
    name=name
) 

async def run():
      
    filtered_view = dataset.select_fields("ground_truth")
    
    print("Loading dataset into FiftyOne...")    
    session = fo.launch_app(filtered_view, browser=True, auto=False)
    dataset.persistent=True    
    
await run()


In [ ]:
import fiftyone.brain as fob
import fiftyone.zoo as foz

# Compute embeddings for all samples
fob.compute_visualization(
    dataset,
    embeddings="default_embedding",
    brain_key="arcade_emb"
)

In [ ]:
# Specify the field containing the patches (e.g., detections)
patches_field = "ground_truth"

# Option 1: Use a pre-trained model from the FiftyOne Model Zoo
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")

# Compute embeddings for the patches using the specified model
dataset.compute_patch_embeddings(
    patches_field=patches_field, 
    model=model, 
    embeddings_field="patches_embedding",
    num_workers=0
    )


In [ ]:
fob.compute_visualization(
    dataset,
    patches_field="ground_truth",             # 👈 your field
    embeddings="patches_embedding",            # 👈 patch embedding field
    brain_key="segmentations_patch_embeddings",# 👈 name for this embedding run
    num_workers=0
)

In [ ]:
dataset.reload()
print(dataset)

In [ ]:
import webbrowser

webbrowser.open("http://localhost:5151/")